In [1]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForTokenClassification
from pathlib import Path
import numpy as np
import torch
from tokenizers import AddedToken
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import pandas as pd
from datasets import Dataset

kaggle=False

path="/kaggle/input/pii-detection-removal-from-educational-data" if kaggle else "data"
train_path = path + "/train.json"
test_path = path + "/test.json"

mixtral_path="data/mpware_mixtral8x7b_v1.1.json" if not kaggle else "/kaggle/input/mixtral-8x7b-v11/mixtral8x7b_v1.1.json"

model_path = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-base" if kaggle else "microsoft/deberta-v3-base"

if not kaggle: import neptune
if not kaggle: from seqeval.metrics import recall_score, precision_score, f1_score, accuracy_score

https://www.kaggle.com/datasets/mpware/pii-mixtral8x7b-generated-essays

In [2]:
cross_entropy_weight_multi = 400

CROSS_ENTROPY_WEIGHTS = [cross_entropy_weight_multi]*12
CROSS_ENTROPY_WEIGHTS.append(1)


# best PII-265

parameter= {
    "model": model_path,
    "max_length": 1024,
    "inference_max_length": 2000,
    "batch_size": 4,
    "inference_batch_size": 1,
    "lr": 5e-05,
    "lr_scale_unfreeze": 0.01,
    "filter_no_pii_percent_allow": 0.2,
    "notebook": "20_deberta base_1024len.ipynb",
    "CROSS_ENTROPY_WEIGHT_MULTI": cross_entropy_weight_multi,
    "epochs_before_unfreeze": 1,
    "epochs_after_unfreeze": 2,
    "train_test_split": 0.2,
    "num_proc": 16 if not kaggle else 2, 
    "freeze_embeddings": False,
    "freeze_layers": 6,
    "warumup_steps": 500,
    "weight_decay": 0.01,
    "logging_dir": './logs',
    "logging_steps": 10,
    "evaluation_strategy": "steps",
    "eval_steps": 400,
    "save_steps": 400,
    "save_total_limit": 3,
    "load_best_model_at_end": False,
    "metric_for_best_model": "f1",
    "greater_is_better": True,
    "overwrite_output_dir": True,
    "report_to": "none",
}

print(parameter["lr"]*parameter["lr_scale_unfreeze"])

5.000000000000001e-07


In [3]:
target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

In [4]:
from itertools import chain
import json

data = json.load(open(train_path))
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

In [5]:
import random

def tokenize(example, tokenizer, label2id, max_length, all_labels_list):
    text = []
    import numpy as np

    # these are at the character level
    labels = []
    targets = []

    for t, l, ws in zip(example["tokens"], example["labels"], example["trailing_whitespace"]):

        text.append(t)
        labels.extend([l]*len(t))
        
        if l in all_labels_list:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
    
    target_num = sum(targets)
    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:

        # CLS token
        if start_idx == 0 and end_idx == 0: 
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        try:
            token_labels.append(label2id[labels[start_idx]])
        except:
            token_labels.append(label2id["O"])

    length = len(tokenized.input_ids)

    return {
        **tokenized,
        "labels": token_labels,
        "length": length,
        "target_num": target_num,
        "group": 1 if target_num>0 else 0
    }

# https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/468844
def filter_no_pii(example, percent_allow=parameter["filter_no_pii_percent_allow"]):
    # Return True if there is PII
    # Or 20% of the time if there isn't
    has_pii = set("O") != set(example["labels"])
    return has_pii or (random.random() < percent_allow)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
tokenizer.add_tokens(AddedToken("\n", normalized=False)) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [7]:
len_data=len(data)
valid_idx = random.sample(range(len_data), int(len_data*parameter["train_test_split"]))
train_idx = list(set(range(len_data)) - set(valid_idx))

train_data = [data[i] for i in train_idx]
valid_data = [data[i] for i in valid_idx]




mixtral_data=json.load(open(mixtral_path))
dict_mixtral={
    "full_text": [x["full_text"] for x in mixtral_data],
    "document": [str(x["document"]) for x in mixtral_data],
    "tokens": [x["tokens"] for x in mixtral_data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in mixtral_data],
    "labels": [x["labels"] for x in mixtral_data],
}
data = json.load(open(train_path))
dict_train={
    "full_text": [x["full_text"] for x in train_data],
    "document": [str(x["document"]) for x in train_data],
    "tokens": [x["tokens"] for x in train_data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in train_data],
    "labels": [x["labels"] for x in train_data],
}

dict_valid={
    "full_text": [x["full_text"] for x in valid_data],
    "document": [str(x["document"]) for x in valid_data],
    "tokens": [x["tokens"] for x in valid_data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in valid_data],
    "labels": [x["labels"] for x in valid_data],
}

full_data = {
    "full_text": dict_train["full_text"] + dict_mixtral["full_text"],
    "document": dict_train["document"] + dict_mixtral["document"],
    "tokens": dict_train["tokens"] + dict_mixtral["tokens"],
    "trailing_whitespace": dict_train["trailing_whitespace"] + dict_mixtral["trailing_whitespace"],
    "labels": dict_train["labels"] + dict_mixtral["labels"],
}

In [8]:
train_ds = Dataset.from_dict(full_data)

train_ds = train_ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": parameter["max_length"], "all_labels_list": target}, num_proc=parameter["num_proc"])
train_ds=train_ds.filter(filter_no_pii, num_proc=parameter["num_proc"])
valid_ds = Dataset.from_dict(dict_valid)
valid_ds = valid_ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": parameter["max_length"], "all_labels_list": target}, num_proc=parameter["num_proc"])


print(len(train_ds), len(valid_ds))


Map (num_proc=16):   0%|          | 0/8138 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/8138 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1361 [00:00<?, ? examples/s]

8138 1361


In [9]:
def tokenize_inference(example, tokenizer, max_length):
        text = []
        for t,  ws in zip(example["tokens"], example["trailing_whitespace"]):
            text.append(t)
            if ws:
                text.append(" ")
        tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
        text = "".join(text)
        length = len(tokenized.input_ids)
        return {
            **tokenized,
            "length": length,
        }
        
class TestTokenizer():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def preprocess(self, example):
        # Preprocess the tokens and labels by adding trailing whitespace and labels
        tokens = []
        tokens_without_ws = []
        token_map = [] # Use the index as labels
        index = 0
        for token, t_ws in zip(example["tokens"], example["trailing_whitespace"]):
            tokens_without_ws.append(token)
            tokens.append(token)
            token_map.extend([index] * len(token))
            # Added trailing whitespace and label if true and 
            if t_ws:
                tokens.append(" ")
                token_map.append(-1)
            index += 1
        return tokens, token_map, tokens_without_ws
    
    def tokenize(self, example):
        tokens, token_map, tokens_without_ws = self.preprocess(example)
        text = "".join(tokens)
        tokenized = self.tokenizer(text, return_offsets_mapping=True, padding="max_length",
                                   truncation=True, max_length=parameter["inference_max_length"])
        return {**tokenized, "token_map": token_map, "tokens": tokens, "tokens_without_ws": tokens_without_ws} 

class PiiDatasetInference(torch.utils.data.Dataset):
        def __init__(self, dataset, tokenizer):
            self.dataset = dataset
            self.tokenizer=TestTokenizer(tokenizer)
            
        def __getitem__(self, idx):
            vals=self.tokenizer.tokenize(self.dataset[idx])
            input_ids = torch.tensor(vals["input_ids"])
            attention_mask = torch.tensor(vals["attention_mask"])
            document_id = self.dataset[idx]["document"]
            return input_ids, attention_mask, document_id, vals
        
        def __len__(self):
            return len(self.dataset)

# Convert preds to a list of dictionaries
def to_test_submission(preds=None, dataset=None, document_ids=None, id2label=None):
    pairs = []
    row_id = 0
    results = []
    
    for i in range(len(preds)):
        input_ids, attention_mask, document_id, vals = dataset[i]
        token_map=vals["token_map"]
        offsets=vals["offset_mapping"]
        tokens=vals["tokens_without_ws"]
        #print("tokens", tokens)
        pred=preds[i]
        #print("original_text", original_text)
        #print("token_map", token_map)
        #print("offsets", offsets)   
        #print("pred", pred)


        for token_pred, input_id, (start_idx, end_idx) in zip(pred, input_ids, offsets):
            #print("\nnow doing ", start_idx,  end_idx, token_pred)
            if start_idx == 0 and end_idx == 0: # Skip 0 offset
                continue
            # Skip spaces 
            while start_idx < len(token_map):
                #print("loop, start_idx now", start_idx) 
                #print(" tokens[token_map[start_idx]]: ", tokens[token_map[start_idx]] if not tokens[token_map[start_idx]].isspace() else "WHITESPACE")          
                if token_map[start_idx] == -1: # Skip unknown tokens               
                    start_idx += 1
                elif tokens[token_map[start_idx]].isspace(): # Skip white space
                    start_idx += 1
                else:
                    break
            # Ensure start index < length
            if start_idx < len(token_map):
                token_id = token_map[start_idx]
                #print("token_id", token_id)
                #token_id= input_id.item()
                label_pred = id2label[token_pred.item()]
                #print("label_pred", label_pred)
                # ignore "O" and whitespace preds
                if label_pred != "O" and token_id != -1:
                    #print("is PII", token_id, label_pred)
                    token_str = tokens[token_id]
                    pair=(document_id, token_id)
                    if pair not in pairs:
                        results.append({
                            "row_id": row_id, 
                            "document": document_id,
                            "token": token_id, 
                            "label": label_pred,
                            "token_str": token_str
                        })
                        pairs.append(pair)
                        row_id += 1

    # Create a dataframe 
    return results

def create_submission(model, filename="submission.csv"):
    data = json.load(open(train_path))
    from itertools import chain
    all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
    label2id = {l: i for i,l in enumerate(all_labels)}
    id2label = {v:k for k,v in label2id.items()}

    data=json.load(open(test_path))
    tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
    my_dataset=PiiDatasetInference(data, tokenizer)
    loader=torch.utils.data.DataLoader(my_dataset, batch_size=1, shuffle=False)

    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    # stack all predictions into tensor
    all_preds = []

    for id, attention_mask, document_ids, vals in loader:
        id=id.to(device)
        attention_mask=attention_mask.to(device)
        preds=model(id, attention_mask).get('logits').argmax(dim=2)
        all_preds.append(preds)
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
                #print(f"Document: {document_id.item()} TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
        #        output[row_id]={"document":document_id.item(), "token":id.item(), "label":id2label[pred.item()]}
        #        row_id+=1
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
        #        print(f"TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
    
   
    all_preds = torch.cat(all_preds, dim=0)
    
    results = to_test_submission(preds=all_preds, dataset=my_dataset, document_ids=document_ids, id2label=id2label)
    if len(results) == 0:
        print("Error in create_submission(): No predictions made, probably because the model is not learning. Check the model and the data.")
        return
    df = pd.DataFrame(results)
    df=df[["row_id", "document", "token", "label"]]
    print(df)
    df.to_csv(filename, index=False)

#create_submission(MyModel(parameter['model'], len(label2id)).to(device), "submission_just_dumb.csv")
# create_submission(model, "submission.csv")
    




In [10]:
from transformers import DataCollatorForTokenClassification

collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

In [11]:
# using Trainer and TrainingArguments from transformers


def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

from functools import partial
from transformers import TrainerCallback
from transformers.trainer_callback import TrainerControl, TrainerState
from transformers.training_args import TrainingArguments

def get_trainer(model, train_dataloader, valid_dataloader, learnrate_multiplier=1.0):

    if not kaggle:
        from transformers.integrations import NeptuneCallback

        run = neptune.init_run(
            project="bernd.heidemann/PII",
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNjBlYzVkNi0zZTUwLTQ1ODYtODhlNC02NDUxNDg0MDdjNzUifQ==",
        )  # your credentials
        run["parameters"] = {
        **parameter
        }

        neptune_callback = NeptuneCallback(run=run, log_model_weights=False, log_parameters=False)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=parameter["epochs_before_unfreeze"]+parameter["epochs_after_unfreeze"],
        per_device_train_batch_size=parameter["batch_size"],  # batch size per device during training
        per_device_eval_batch_size=parameter["inference_batch_size"],   # batch size for evaluation
        warmup_steps=parameter["warumup_steps"],                # number of warmup steps for learning rate scheduler
        weight_decay=parameter["weight_decay"],               # strength of weight decay
        logging_dir=parameter["logging_dir"],            # directory for storing logs
        logging_steps=parameter["logging_steps"],
        evaluation_strategy=parameter["evaluation_strategy"],
        eval_steps=parameter["eval_steps"],
        save_steps=parameter["save_steps"],
        save_total_limit=parameter["save_total_limit"],
        load_best_model_at_end=parameter["load_best_model_at_end"],
        metric_for_best_model="f1" if not kaggle else "eval_loss",
        greater_is_better=True if not kaggle else False,
        overwrite_output_dir=parameter["overwrite_output_dir"],
        report_to=parameter["report_to"],
        learning_rate=parameter["lr"]
    )

    class FreezingCallback(TrainerCallback):
        def on_epoch_begin(self, args, state, control, model, **kwargs):
            if state.epoch == parameter["epochs_before_unfreeze"]:
                # change learning rate
                optimizer= kwargs["optimizer"]

                for param_group in optimizer.param_groups:
                    param_group['lr'] = parameter["lr"]*parameter["lr_scale_unfreeze"]
                for param in model.base_model.parameters():
                    param.requires_grad = True
                
    class MyTrainer(Trainer):
        def __init__(self, model=None, args=None, train_dataset=None, eval_dataset=None, compute_metrics=None, callbacks=None):
            super().__init__(model=model, args=args, train_dataset=train_dataset, eval_dataset=eval_dataset, compute_metrics=compute_metrics, callbacks=callbacks)
            # Definieren Sie hier Ihre Gewichte für die Klassen, z.B. torch.tensor([1.0, 2.0, 0.5])
            self.weight = torch.tensor(CROSS_ENTROPY_WEIGHTS).to(device)
            self.loss_func=torch.nn.CrossEntropyLoss(ignore_index=-100, weight=torch.tensor(CROSS_ENTROPY_WEIGHTS, dtype=torch.float32).to(device))

        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get('logits')
            loss = self.loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss
        
    trainer = MyTrainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataloader,         # training dataset
        eval_dataset=valid_dataloader,             # evaluation dataset
        compute_metrics=partial(compute_metrics, all_labels=all_labels) if not kaggle else None,
        callbacks=[neptune_callback, FreezingCallback()] if not kaggle else [FreezingCallback()]
    )
    return trainer

In [12]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"



model = AutoModelForTokenClassification.from_pretrained(
    parameter["model"],
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

if parameter['freeze_embeddings']:
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
        
if parameter['freeze_layers'] > 0:
    for layer in model.deberta.encoder.layer[:parameter['freeze_layers']]:
        for param in layer.parameters():
            param.requires_grad = False

#my_model=MyModel(parameter['model'], len(label2id))
# set torch seed
torch.manual_seed(189237)
trainer=get_trainer(model, train_ds, valid_ds)
trainer.train()

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\neptune\common\warnings.py:71: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  warnings.warn(


https://app.neptune.ai/bernd.heidemann/PII/e/PII-266


c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/16280 [00:00<?, ?it/s]

{'loss': 3.1686, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 3.0549, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 2.9252, 'learning_rate': 3e-06, 'epoch': 0.01}
{'loss': 2.6947, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 2.4117, 'learning_rate': 5e-06, 'epoch': 0.02}
{'loss': 2.2911, 'learning_rate': 6e-06, 'epoch': 0.03}
{'loss': 2.1482, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.03}
{'loss': 1.621, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.04}
{'loss': 1.7245, 'learning_rate': 9e-06, 'epoch': 0.04}
{'loss': 1.5878, 'learning_rate': 1e-05, 'epoch': 0.05}
{'loss': 0.9594, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.05}
{'loss': 0.9283, 'learning_rate': 1.2e-05, 'epoch': 0.06}
{'loss': 1.1425, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.06}
{'loss': 0.7517, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.07}
{'loss': 1.1812, 'learning_rate': 1.5e-05, 'epoch': 0.07}
{'loss': 0.895, 'learning

...

  0%|          | 0/1361 [00:00<?, ?it/s]

{'eval_loss': 0.030603989958763123, 'eval_recall': 0.9341126461211477, 'eval_precision': 0.8363463368220743, 'eval_f1': 0.9299316406250001, 'eval_runtime': 41.0508, 'eval_samples_per_second': 33.154, 'eval_steps_per_second': 33.154, 'epoch': 7.67}
{'loss': 0.0, 'learning_rate': 2.122940430925222e-06, 'epoch': 7.67}
{'loss': 0.0, 'learning_rate': 2.091254752851711e-06, 'epoch': 7.68}
{'loss': 0.0001, 'learning_rate': 2.0595690747782005e-06, 'epoch': 7.68}
{'loss': 0.0, 'learning_rate': 2.0278833967046895e-06, 'epoch': 7.69}
{'loss': 0.0001, 'learning_rate': 1.996197718631179e-06, 'epoch': 7.69}
{'loss': 0.0, 'learning_rate': 1.964512040557668e-06, 'epoch': 7.7}
{'loss': 0.0002, 'learning_rate': 1.9328263624841573e-06, 'epoch': 7.7}
{'loss': 0.0009, 'learning_rate': 1.9011406844106463e-06, 'epoch': 7.71}
{'loss': 0.0, 'learning_rate': 1.8694550063371357e-06, 'epoch': 7.71}
{'loss': 0.0003, 'learning_rate': 1.837769328263625e-06, 'epoch': 7.71}
{'loss': 0.0, 'learning_rate': 1.80608365019

  0%|          | 0/1361 [00:00<?, ?it/s]

{'eval_loss': 0.02844315953552723, 'eval_recall': 0.9351753453772582, 'eval_precision': 0.8255159474671669, 'eval_f1': 0.9304216989955676, 'eval_runtime': 41.0685, 'eval_samples_per_second': 33.14, 'eval_steps_per_second': 33.14, 'epoch': 7.86}
{'loss': 0.0, 'learning_rate': 8.55513307984791e-07, 'epoch': 7.87}
{'loss': 0.0001, 'learning_rate': 8.238276299112802e-07, 'epoch': 7.87}
{'loss': 0.0001, 'learning_rate': 7.921419518377694e-07, 'epoch': 7.88}
{'loss': 0.0294, 'learning_rate': 7.604562737642586e-07, 'epoch': 7.88}
{'loss': 0.0, 'learning_rate': 7.287705956907478e-07, 'epoch': 7.89}
{'loss': 0.0, 'learning_rate': 6.97084917617237e-07, 'epoch': 7.89}
{'loss': 0.0112, 'learning_rate': 6.653992395437262e-07, 'epoch': 7.9}
{'loss': 0.0001, 'learning_rate': 6.337135614702154e-07, 'epoch': 7.9}
{'loss': 0.0001, 'learning_rate': 6.020278833967048e-07, 'epoch': 7.91}
{'loss': 0.0001, 'learning_rate': 5.70342205323194e-07, 'epoch': 7.91}
{'loss': 0.0, 'learning_rate': 5.386565272496832e

TrainOutput(global_step=16280, training_loss=0.03680243135802396, metrics={'train_runtime': 6670.4371, 'train_samples_per_second': 9.76, 'train_steps_per_second': 2.441, 'train_loss': 0.03680243135802396, 'epoch': 8.0})

In [14]:
create_submission(model, f"submission.csv")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


    row_id  document  token           label
0        0         7      9  B-NAME_STUDENT
1        1         7     10  I-NAME_STUDENT
2        2         7    482  B-NAME_STUDENT
3        3         7    483  I-NAME_STUDENT
4        4         7    741  B-NAME_STUDENT
5        5         7    742  I-NAME_STUDENT
6        6        10      0  B-NAME_STUDENT
7        7        10      1  I-NAME_STUDENT
8        8        10    464  B-NAME_STUDENT
9        9        10    465  I-NAME_STUDENT
10      10        16      4  B-NAME_STUDENT
11      11        16      5  I-NAME_STUDENT
12      12        20      5  B-NAME_STUDENT
13      13        20      6  I-NAME_STUDENT
14      14        20      8  I-NAME_STUDENT
15      15        56     12  B-NAME_STUDENT
16      16        56     13  I-NAME_STUDENT
17      17        86      6  B-NAME_STUDENT
18      18        86      7  I-NAME_STUDENT
19      19        93      0  B-NAME_STUDENT
20      20        93      1  I-NAME_STUDENT
21      21       104      7  B-N

In [ ]:
# save trainer
trainer.save_model("final_model")